In [1]:
import pandas as pd
import recordlinkage
from graphdatascience import GraphDataScience
from getpass import getpass
from time import perf_counter

In [2]:
email_df = pd.read_csv("data/emails", header=None)
email_df.columns=['email']

In [3]:
email_df['first_letters'] = email_df['email'].str[:2]

In [4]:
email_df['tld'] = email_df['email'].str.split(".").map(lambda x: x[-1])

In [5]:
email_df['middle_letters'] = email_df['email'].str[2:].str.split(".").map(lambda x: ".".join(x[:-1]))

In [6]:
email_df.head()

,email,first_letters,tld,middle_letters
0,rhonda82@yahoo.com,rh,com,onda82@yahoo
1,mvelazquez@hotmail.com,mv,com,elazquez@hotmail
2,sandrawilliams@chavez-morales.com,sa,com,ndrawilliams@chavez-morales
3,caleb08@pacheco.com,ca,com,leb08@pacheco
4,esmith@chen.com,es,com,mith@chen


In [7]:
start_time = perf_counter()
indexer = recordlinkage.Index()
indexer.block(["first_letters", "tld"])
indexer.block("middle_letters")
candidate_links = indexer.index(email_df)
end_time = perf_counter()

In [8]:
len(candidate_links)

29901916

In [9]:
print(end_time - start_time)

12.798818184004631


In [10]:
start_time = perf_counter()
compare = recordlinkage.Compare()
compare.string(
    "email", "email", method="levenshtein", label="email", threshold=0.8
)
features = compare.compute(candidate_links, email_df)
end_time = perf_counter()

In [11]:
print(end_time - start_time)

267.308042293007


In [12]:
features[features['email'] == 1].shape

(260295, 1)

In [13]:
features[features['email'] == 1].head()

email
238 199    1.0
253 201    1.0
301 201    1.0
    253    1.0
309 38     1.0

In [14]:
email_df.iloc[238]

email             robin08@hotmail.com
first_letters                      ro
tld                               com
middle_letters          bin08@hotmail
Name: 238, dtype: object

In [15]:
email_df.iloc[199]

email             robert05@hotmail.com
first_letters                       ro
tld                                com
middle_letters          bert05@hotmail
Name: 199, dtype: object

In [4]:
neo4j_password = getpass("neo4j password")

neo4j password ········


In [ ]:
gds = GraphDataScience(